## Import packages

In [1]:
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint
from CFModel import CFModel

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN 5006)


In [2]:
CVSC_TRAIN_CSV_FILE = 'fb15k_cvsc_train.csv'
MODEL_WEIGHTS_FILE = 'fb15k_cvsc_weights.h5'
RNG_SEED = 1446557
K_FACTORS = 50

## Load CVSC training triples

In [3]:
triples = pd.read_csv(CVSC_TRAIN_CSV_FILE, sep='\t', usecols=['pid', 'rid']).sample(frac=1., random_state=RNG_SEED)

In [4]:
n_pairs = triples['pid'].max() + 1
m_relations = triples['rid'].max() + 1
print n_pairs, 'pairs,', m_relations, 'relations'

2995738 pairs, 26154 relations


## Define model

In [5]:
model = CFModel(n_pairs, m_relations, K_FACTORS)
model.compile(loss='mse', optimizer='adamax')
model.load_weights(MODEL_WEIGHTS_FILE)

## Train model

In [6]:
callbacks = [EarlyStopping('val_loss', patience=2), ModelCheckpoint(MODEL_WEIGHTS_FILE, save_best_only=True)]
model.fit([triples['pid'].values, triples['rid'].values], np.ones(len(triples)), 
             nb_epoch=10, validation_split=.1, verbose=1, callbacks=callbacks)

Train on 6185264 samples, validate on 687252 samples
Epoch 1/10
6185264/6185264 [==============================] - 16246s - loss: 0.2084 - val_loss: 0.4326
Epoch 2/10
6185264/6185264 [==============================] - 16242s - loss: 0.1481 - val_loss: 0.4043
Epoch 3/10
2881184/6185264 [============>.................] - ETA: 8669s - loss: 0.1086

KeyboardInterrupt: 